#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

./bin/pyspark --conf "spark.mongodb.input.uri=mongodb://localhost:27017/AIT614.NYCSpeed?readPreference=primaryPreferred" 
              --conf "spark.mongodb.output.uri=mongodb://localhost:27017/AIT614.NYCSpeed" 
              --packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1

In [2]:
import findspark
findspark.init()
#findspark.add_packages("org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")

In [10]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("spark")\
    .config('spark.driver.memory', '32g')\
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/NYCSpeed.coll") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/NYCSpeed.coll") \
    .config("spark.mongodb.output.database", "AIT614")\
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0")\
    .enableHiveSupport()\
    .getOrCreate()

#mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false

In [6]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())
])

schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
df2 = spark.read.format("csv").option("header", "true").schema(schema).load("file:/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv")
df2 = df2.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df2 = df2.na.drop()
df2 = df2.drop(*("Status","EncodedLinkPoints","EncodedPolyLineLvls","TranscomId"))
df2.createOrReplaceTempView("dataset")


root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)

+---+-----+----------+-------------------+-------+--------------------+--------------------+-------------+--------------------+
| ID|Speed|TravelTime|           timedate| LinkId|          LinkPoints|               Owner|      Borough|           Link_Name|
+---+-----+----------+-------------------+-------+--------------------+--------------------+-------------+--------------------+
|398|42.87|       175|2018-11-17 08:43:40|4456481|40.77223,-73.9199...|MTA Bridges & Tun...|       Queens|TBB S MANHATTAN L...|
|168|49.08|        94|2018-11-17 08:43:40|4456496|40.814761,-73.836...|         NYC_DOT_LIC|        Bronx|BWB S Toll Plaza ...

22/03/22 20:45:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [9]:
df2 = spark.sql("select * from dataset where year(timedate) >= 2017 order by timedate asc")
df2.show()

22/03/22 20:50:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


+---+-----+----------+-------------------+-------+--------------------+-----------+-------------+--------------------+
| ID|Speed|TravelTime|           timedate| LinkId|          LinkPoints|      Owner|      Borough|           Link_Name|
+---+-----+----------+-------------------+-------+--------------------+-----------+-------------+--------------------+
|381| 37.9|        38|2017-01-07 19:08:40|4616194|40.608031,-74.132...|NYC_DOT_LIC|Staten Island|SIE E WOOLEY AVEN...|
|375| 37.9|       150|2017-01-07 19:08:40|4616193|40.61052,-74.0976...|NYC_DOT_LIC|Staten Island|SIE E BRADLEY AVE...|
|435| 37.9|       142|2017-01-07 19:08:40|4616198|40.62102,-74.1688...|NYC_DOT_LIC|Staten Island|WSE N-SIE E SOUTH...|
|378| 37.9|        66|2017-01-07 19:08:40|4616197|40.6210105,-74.16...|NYC_DOT_LIC|Staten Island|SIE E SOUTH AVENU...|
|350| 29.2|       248|2017-01-07 19:08:40|4616196|40.63092,-74.1459...|NYC_DOT_LIC|Staten Island|MLK S - SIE E WAL...|
|376| 37.9|       131|2017-01-07 19:08:40|461619

In [11]:
df2.write.format("mongo").mode('overwrite').save()

22/03/22 20:54:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv
22/03/22 20:55:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Spe

In [32]:
columns1= "year(timedate), month(timedate), day(timedate), hour(timedate), minute(timedate), second(timedate)"
select1 = "select "+columns1+", count(second(timedate))"
from1 = " from dataset"
groupby2= " group by "+columns1
query1 = select1 + from1 + groupby2

In [33]:
df2.createOrReplaceTempView("dataset")
linkpoints = spark.sql(query1)
linkpoints.show()

22/03/13 00:30:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DATA_AS_OF
 Schema: timedate
Expected: timedate but found: DATA_AS_OF
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


+--------------+---------------+-------------+--------------+----------------+----------------+-----------------------+
|year(timedate)|month(timedate)|day(timedate)|hour(timedate)|minute(timedate)|second(timedate)|count(second(timedate))|
+--------------+---------------+-------------+--------------+----------------+----------------+-----------------------+
|          2018|             11|           17|            10|              52|              40|                      7|
|          2018|             11|           17|            11|              12|              43|                      7|
|          2018|             11|           17|            12|              48|              16|                      7|
|          2018|             11|           17|            12|              48|              40|                      6|
|          2019|              6|            6|             1|              53|               6|                      7|
|          2018|             11|        

In [34]:
linkpoints.write.format('csv').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/second.csv')

22/03/13 00:31:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DATA_AS_OF
 Schema: timedate
Expected: timedate but found: DATA_AS_OF
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [5]:
spark.stop()